In [ ]:
import cv2
import numpy as np
import math
import os
import time
import func
import cv2
import pyrealsense2
from realsense_camera import *
from kuka import Kuka
from openshowvar import *
from PIL import Image

'''
#Connect robot
robot = openshowvar(ip = '192.168.1.2', port = 7000)
kuka = Kuka(robot)

print('green button')
kuka.set_base(8)
kuka.set_tool(10)

# Set speed (%)
kuka.set_speed(30)
print('go')

kuka.read_cartessian()
x0=65.26
y0=92.21
z0=149.58
# Построение траектории
trajectory_arr = []
# Построение квадрата
trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
trajectory_arr.append(np.array([x0, y0, z0, 90, 0, 180]))
trajectory_arr = np.array(trajectory_arr)
# Отправка траектории роботу
kuka.lin_continuous(trajectory_arr)
'''

MIN_MATCH_COUNT=10

# matrix of camera parameters (made up but works quite well for me) 
detector=cv2.SIFT_create()

FLANN_INDEX_KDITREE=0
flannParam=dict(algorithm=FLANN_INDEX_KDITREE,tree=5)
flann=cv2.FlannBasedMatcher(flannParam,{})

#classes = ["simple.jpg", "simple2.png", "simple3.jpg"]
trainImg=cv2.imread("simple6.jpg",0)
trainKP,trainDesc=detector.detectAndCompute(trainImg,None)
axis_length=50
cam=RealsenseCamera()
#cam=cv2.VideoCapture(0)

while True:
    ret, QueryImgBGR, depth=cam.get_frame_stream()
    #ret, QueryImgBGR=cam.read()
    h,w=QueryImgBGR.shape[:2]
    QueryImg=cv2.cvtColor(QueryImgBGR,cv2.COLOR_BGR2GRAY)
    queryKP,queryDesc=detector.detectAndCompute(QueryImg,None)
    matches=flann.knnMatch(queryDesc,trainDesc,k=2)
    goodMatch=[] 
    for m,n in matches:
        if(m.distance<0.75*n.distance):
            goodMatch.append(m)
    if(len(goodMatch)>MIN_MATCH_COUNT):
        tp=[]
        qp=[]
        for m in goodMatch:
            tp.append(trainKP[m.trainIdx].pt)
            qp.append(queryKP[m.queryIdx].pt)
        tp,qp=np.float32((tp,qp))
        H,status=cv2.findHomography(tp,qp,cv2.RANSAC,3.0)
        h,w=trainImg.shape
        trainBorder=np.float32([[[0,0],[0,h-1],[w-1,h-1],[w-1,0]]])
        queryBorder=cv2.perspectiveTransform(trainBorder,H)
        queryBorder=np.asarray(queryBorder).astype(int)
        point_center=func.calculate_center(queryBorder)
        print(queryBorder)
        point_center=np.asarray(point_center).astype(int)
        '''
        kuka.read_cartessian()
        X_mm=kuka.x_cartessian+((476-point_center[0])*1.35)
        Y_mm=-(kuka.y_cartessian+((145-point_center[1])*1.35))

        kuka.read_cartessian()
        trajectory_arr = []
        trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
        trajectory_arr.append(np.array([X_mm, Y_mm, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, -180]))
        trajectory_arr = np.array(trajectory_arr)
        kuka.lin_continuous(trajectory_arr)
        '''
        
        #corners=func.calculate_rectangle_opposite_corners(queryBorder)
        #depth_object,angle=func.calculate_rotation_z(cam,queryBorder)
        #x=int((corners[0][0]+corners[1][0])/2)
        #x1=corners[1][0]
        #y1=corners[1][1]
        #y=int((corners[0][1]+corners[1][1])/2)

        rotation=func.calculate_rotation(queryBorder)
        print(rotation)
        cv2.polylines(QueryImgBGR,[np.int32(queryBorder)],True,(0,255,0),2)
        cv2.circle(QueryImgBGR, point_center, 6, (118, 103, 154), 6)

        
        #cv2.circle(QueryImgBGR, queryBorder[0][0], 6, (118, 103, 154), 6)
        #cv2.circle(QueryImgBGR, queryBorder[0][1], 6, (118, 103, 154), 6)
        #cv2.circle(QueryImgBGR, queryBorder[0][2], 6, (118, 103, 154), 6)
        #cv2.circle(QueryImgBGR, queryBorder[0][3], 6, (118, 103, 154), 6)
        
        #cv2.line(QueryImgBGR, (corners[0][0], int((corners[0][1]+corners[1][1])/2)), (corners[1][0], int((corners[0][1]+corners[1][1])/2)), (255, 0, 0), 3)
    
        #cv2.circle(QueryImgBGR, (corners[0][0]+60, corners[0][1]+30), 6, (255, 0, 0), 6)
        #cv2.circle(QueryImgBGR, (corners[1][0]-30, corners[0][1]+30), 6, (0, 255, 0), 6)
        #cv2.circle(QueryImgBGR, (corners[0][0]+30, corners[1][1]-30), 6, (0, 0, 255), 6)
        #cv2.circle(QueryImgBGR, (corners[1][0]-30, corners[1][1]-30), 6, (0, 0, 255), 6)
        #cv2.putText(QueryImgBGR, str(angle), point_center, cv2.FONT_HERSHEY_SIMPLEX, 1, (118, 103, 154), 2, cv2.LINE_AA)
        
        #cv2.circle(depth_colormap, queryBorder[0][0], 6, (118, 103, 154), 6)
        #cv2.circle(depth_colormap, queryBorder[0][1], 6, (118, 103, 154), 6)
        #cv2.circle(depth_colormap, queryBorder[0][2], 6, (118, 103, 154), 6)
        #cv2.circle(depth_colormap, queryBorder[0][3], 6, (118, 103, 154), 6)
        #ima = np.hstack((QueryImgBGR, depth_colormap))
        #mean_x, mean_y=func.key_points2(QueryImgBGR, corners, queryBorder)
        #cv2.circle(QueryImgBGR,(int(mean_x),int(mean_y)), 6, (118, 103, 154), 6)
        
        
        #mask = np.zeros(QueryImgBGR.shape,np.uint8)
        #mask[corners[0][1]:corners[1][1],corners[0][0]:corners[1][0]] = QueryImgBGR[corners[0][1]:corners[1][1],corners[0][0]:corners[1][0]]

        #pts = cv2.KeyPoint_convert(queryKP) 
        #a=cv2.KeyPoint(pts[0],pts[1])
        #print(a)
        #height, width = QueryImgBGR.shape[:2] 
        #center = (width/2, height/2) 
        #rotat_mat=cv2.getRotationMatrix2D(center,180-rotation,0.7)
        #rotate_im=cv2.warpAffine(mask,rotat_mat, dsize=(width, height))
        #queryKP,queryDesc=detector.detectAndCompute(rotate_im,None)
        #cv2.drawKeypoints(rotate_im, queryKP, rotate_im, (0, 255, 255), flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
        '''
        kuka.read_cartessian()
        trajectory_arr = []
        trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
        trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, 90, angle, -180]))
        trajectory_arr = np.array(trajectory_arr)
        kuka.lin_continuous(trajectory_arr)
        time.sleep(5)
        kuka.read_cartessian()
        trajectory_arr = []
        trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
        trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, 90, 0, -180]))
        trajectory_arr = np.array(trajectory_arr)
        kuka.lin_continuous(trajectory_arr)
        '''
    else:
        print ("Not Enough match found- %d/%d"%(len(goodMatch),MIN_MATCH_COUNT))
    cv2.imshow('result',QueryImgBGR)
    if cv2.waitKey(10)==ord('q'):
        break
cam.release()
cv2.destroyAllWindows()

Not Enough match found- 10/10
[[[587 458]
  [600 477]
  [595 479]
  [601 479]]]
69.14554196042184
[[[449 445]
  [183 119]
  [496 463]
  [448 470]]]
52.391183476578384
Not Enough match found- 10/10
Not Enough match found- 10/10
[[[404 211]
  [424 219]
  [399 206]
  [382 207]]]
-154.35899417569482
[[[416 208]
  [654 366]
  [474  17]
  [ 73 122]]]
-126.4163785198861
Not Enough match found- 8/10
Not Enough match found- 9/10
[[[-237  288]
  [ 605  498]
  [ 621  483]
  [ 579  478]]]
12.104347131238908
[[[314 413]
  [812 654]
  [645 490]
  [574 480]]]
-46.20604877921998
[[[520 458]
  [609 474]
  [724 494]
  [619 476]]]
10.304846468766044
[[[405 211]
  [422 220]
  [398 205]
  [181 120]]]
-147.99461679191648
Not Enough match found- 10/10
[[[185 399]
  [577 467]
  [616 475]
  [563 465]]]
9.904183212973898
Not Enough match found- 7/10
Not Enough match found- 10/10
[[[489 616]
  [327 282]
  [397 154]
  [390 289]]]
-95.06801812660518
[[[404 211]
  [183 225]
  [417 206]
  [416 213]]]
-2.948250721932

C:\Users\djnik\disser\Дескрипторы\func.py:52: RuntimeWarning: invalid value encountered in double_scalars
  angle = np.arccos(x/y)


[[[404 211]
  [419 213]
  [404 211]
  [404 211]]]
-172.40535663140562
[[[404 211]
  [182 225]
  [417 206]
  [416 213]]]
-2.935673446421223
Not Enough match found- 8/10
Not Enough match found- 9/10
[[[449 445]
  [184 120]
  [497 463]
  [448 470]]]
52.41526604188748
Not Enough match found- 9/10
[[[404 211]
  [424 219]
  [399 206]
  [381 206]]]
-152.52556837372254
Not Enough match found- 9/10
Not Enough match found- 9/10
[[[ 438  454]
  [-117  351]
  [1408 1129]
  [1307  462]]]
4.457160997643616
Not Enough match found- 10/10
Not Enough match found- 9/10
Not Enough match found- 9/10
[[[449 445]
  [184 120]
  [496 463]
  [447 470]]]
52.520313750380986
[[[409 183]
  [397 234]
  [401 243]
  [400 231]]]
95.94686305397357
[[[754 693]
  [570 460]
  [599 476]
  [572 448]]]
-125.18648214074543
Not Enough match found- 10/10
[[[404 211]
  [183 225]
  [417 206]
  [416 213]]]
-2.9482507219327325
[[[  497   449]
  [  115   679]
  [-2941  -595]
  [  263   480]]]
-139.62125140201402
[[[343 340]
  [260 23

In [1]:
import cv2
import numpy as np
import math
import os
import time
import func
import cv2
import pyrealsense2
from realsense_camera import *
cam=RealsenseCamera()

def mouse_response(event,x,y,flags,param):
    if event==cv2.EVENT_LBUTTONDOWN:
        print(x,y)
while True:
    ret, QueryImgBGR, depth=cam.get_frame_stream()
    
    cv2.imshow('result',QueryImgBGR)
    cv2.setMouseCallback('result', mouse_response) 
    if cv2.waitKey(10)==ord('q'):
        break
cam.release()
cv2.destroyAllWindows()

476 145


KeyboardInterrupt: 